MobileNet Model on RGB 5G Dataset

The dataset consists of 5 classes:

![Image.png]('F:\FCV\Image.png')

Training Accuracy : 68.87%
Validation Accuracy : 28.4%

In [1]:
# import the libraries as shown below
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
import seaborn as sns
from sklearn.metrics import confusion_matrix
import itertools
from keras.models import load_model
import os
import tensorflow as tf


In [4]:
# re-size all the images to this
IMAGE_SIZE = [224, 224]

train_path = r'F:\\FCV\\data\\Train\\'
valid_path = r'F:\\FCV\\data\\Test\\'

In [5]:
# Import the mobilenet library as shown below and add preprocessing layer to the front of MobileNet
# Here we will be using imagenet weights

mobilenet = MobileNet(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)


17235968/17225924 [==============================] - 8s 0us/step


In [6]:
# don't train existing weights
for layer in mobilenet.layers:
    layer.trainable = False

In [7]:
# useful for getting number of output classes
folders = glob(r'F:\\FCV\\data\\Train\*')

In [8]:
# our layers - you can add more if you want
x = Flatten()(mobilenet.output)
prediction = Dense(len(folders), activation='softmax')(x)

In [9]:
# create a model object
model_mobilenet_RGB = Model(inputs=mobilenet.input, outputs=prediction)

In [10]:
# tell the model what cost and optimization method to use
model_mobilenet_RGB.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [11]:
# Use the Image Data Generator to import the images from the dataset
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True,)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [12]:
# Make sure you provide the same target size as initialied for the image size
training_set = train_datagen.flow_from_directory(train_path,
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory(valid_path,
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')


Found 1500 images belonging to 5 classes.
Found 500 images belonging to 5 classes.


In [13]:
# fit the model
# Run the cell. It will take some time to execute
history_mobilenet_RGB = model_mobilenet_RGB.fit(
  training_set,
  validation_data=test_set,
  epochs=10,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)    

model_mobilenet_RGB.save('mobilenet_RGB_model.h5')
print("model saved to Disk.")

Epoch 1/10
47/47 [==============================] - 141s 3s/step - loss: 6.2038 - accuracy: 0.2720 - val_loss: 2.9490 - val_accuracy: 0.3560
Epoch 2/10
47/47 [==============================] - 119s 3s/step - loss: 3.3124 - accuracy: 0.3820 - val_loss: 3.5211 - val_accuracy: 0.3080
Epoch 3/10
47/47 [==============================] - 119s 3s/step - loss: 2.1490 - accuracy: 0.4807 - val_loss: 2.9193 - val_accuracy: 0.4120
Epoch 4/10
47/47 [==============================] - 121s 3s/step - loss: 2.0135 - accuracy: 0.5407 - val_loss: 3.1280 - val_accuracy: 0.3520
Epoch 5/10
47/47 [==============================] - 117s 2s/step - loss: 1.9424 - accuracy: 0.5587 - val_loss: 3.9676 - val_accuracy: 0.2960
Epoch 6/10
47/47 [==============================] - 117s 2s/step - loss: 1.7652 - accuracy: 0.6293 - val_loss: 4.1388 - val_accuracy: 0.3240
Epoch 7/10
47/47 [==============================] - 117s 2s/step - loss: 1.5653 - accuracy: 0.6573 - val_loss: 4.9396 - val_accuracy: 0.3280
Epoch 8/10
47

In [ ]:
#plot the loss
plt.plot(history_mobilenet_RGB.history['loss'], label='train loss')
plt.plot(history_mobilenet_RGB.history['val_loss'], label='val loss')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.show()

# plot the accuracy
plt.plot(history_mobilenet_RGB.history['accuracy'], label='val acc')
plt.plot(history_mobilenet_RGB.history['val_accuracy'], label='train acc')
plt.legend()
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.show()

In [15]:
# dirname = os.path.join(os.path.dirname(__file__))
model_mobilenet_new_RGB = load_model('mobilenet_RGB_model.h5')
print("model loaded.")

model loaded.


In [16]:
test_generator = ImageDataGenerator(rescale = 1./255)
test_data_generator = test_generator.flow_from_directory(
    valid_path,
    target_size=(224, 224),
    batch_size=32,
    shuffle=False)
test_steps_per_epoch = np.math.ceil(test_data_generator.samples / test_data_generator.batch_size)


Found 500 images belonging to 5 classes.


In [17]:
predictions = model_mobilenet_new_RGB.predict(test_data_generator, steps=test_steps_per_epoch)


In [18]:
# Get most likely class
predicted_classes = np.argmax(predictions, axis=1)

print(test_data_generator.classes)
print(predicted_classes)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3
 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 3 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 

In [19]:
true_classes = test_data_generator.classes
class_labels = list(test_data_generator.class_indices.keys())

In [20]:
report = classification_report(true_classes, predicted_classes, target_names=class_labels)
cmat = confusion_matrix(true_classes, predicted_classes)
acc = accuracy_score(true_classes, predicted_classes)
print(report) 
print(cmat)
print(acc)

              precision    recall  f1-score   support

       16QAM       0.31      0.05      0.09       100
       32QAM       0.24      0.40      0.30       100
        4QAM       0.54      0.29      0.38       100
       64QAM       0.24      0.09      0.13       100
        8QAM       0.26      0.59      0.36       100

    accuracy                           0.28       500
   macro avg       0.32      0.28      0.25       500
weighted avg       0.32      0.28      0.25       500

[[ 5 33 20  8 34]
 [ 0 40  2  7 51]
 [11 26 29  6 28]
 [ 0 34  2  9 55]
 [ 0 33  1  7 59]]
0.284


In [ ]:
confusion_mtx = confusion_matrix(true_classes, predicted_classes) 
# plot the confusion matrix
f,ax = plt.subplots(figsize=(8, 8))
sns.heatmap(confusion_mtx, annot=True, linewidths=0.01,cmap="Greens",linecolor="gray", fmt= '.1f',ax=ax)
plt.xlabel("Predicted Label")
plt.ylabel("True Label")
ax.set_xticklabels(['4QAM_RGB', '8QAM_RGB', '16QAM_RGB', '32QAM_RGB', '64QAM_RGB'])
plt.xticks(rotation=90)
plt.yticks(rotation=0)
ax.set_yticklabels(['4QAM_RGB', '8QAM_RGB', '16QAM_RGB', '32QAM_RGB', '64QAM_RGB'])

plt.title("Confusion Matrix")
plt.show()